In [ ]:
#@title Login to Bank Account
#selenium part to login to banking account (example with commerzbank and microsoft edge)

from selenium import webdriver
#wichtig für edge; anders bei anderen browsern
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
# importiert Action chains; action chains erlauben einen Queu von aufeinanderfolgenden actions
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
#keys wichtig um text einzufügen
from selenium.webdriver.common.keys import Keys


username = "Your username here"
password = "Your password here"


#driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install())) auf der offiziellen webdriver-manager website
#driver = webdriver.Edge(EdgeService(EdgeChromiumDriverManager().install()))# noch fehler, aber diese version fkt irwie teilweise besser; I've been having this problem also since I switched from pip Jupyter to Anaconda Jupyter. This worked for me:driver = webdriver.Chrome(ChromeDriverManager().install())Apparently you don't need the service in the Jupyter package.https://stackoverflow.com/questions/70534875/typeerror-init-got-an-unexpected-keyword-argument-service-error-using-p


driver = webdriver.Edge(EdgeChromiumDriverManager().install()) #es war irwie doppeltgemoppelt. Das funktioniert. EIN HOCH AUF CHATGPT!!! "In the updated code, we directly use the EdgeChromiumDriverManager to handle the driver installation, so there's no need to create an EdgeService object or call install() on it."


driver.get("https://www.commerzbank.de/portal/en/englisch/my-online-banking/my-onlinebanking.html")#opens the edge browser and navigates to the banking Website
driver.implicitly_wait(10) #Waits 5 seconds before throwing an error. Important since cookie window takes time to load, otherwise errors:  https://stackoverflow.com/questions/64032271/handling-accept-cookies-popup-with-selenium-in-python
accept_all = driver.find_element(By.XPATH, value = "//button[@id='uc-btn-accept-banner']")#locates and saves the accept all button to accept the now ubiquitous cookies....still getting errors
login_button = driver.find_element(By.XPATH, value = "//button[@id='headerLoginSubmit']")#locates the login button and saves it as login_button

#driver.set_window_size(1024, 600)#scheint unwichtig, da kein Einfluss auf Endgröße des Bildes; vielleicht rausnehmen?
driver.maximize_window()#maximiert Fenster; wie nach vorne bringen?


# creates an action chain object instance called actions
actions = ActionChains(driver)
actions.click(accept_all)#accepts cookies
actions.click(login_button)#clicks login
actions.perform()#needed at the end to perform action chains; action chains are fired as a queu


login_box = driver.find_element(By.XPATH, value = "//input[@id='teilnehmer']")#creates an object for the login box
password_box = driver.find_element(By.XPATH, value = "//input[@id='pin']")#creates an object for the pin box

login_box.send_keys(username);#fügt den gespeicherten Wert für den Benutzername ein
password_box.send_keys(password);#fügt den gespeicherten Wert für das Passwort ein


login_now = driver.find_element(By.XPATH, value = "//button[@id='loginFormSubmit']")#creates an object for the login box

actions = ActionChains(driver)#create a new instance as we are on a new page now; otherwise we get a staleElement exception
actions.click(login_now)
actions.perform()




tan_box = driver.find_element(By.XPATH, value = "//input[@id='input sc-lsg-textfield']")

tanIn =input('Scanne bitte mit dem Tan Gerät den QR code ein und gib dann den Code in dieses Feld ein\n')
tan_box.send_keys(tanIn);#fügt die eingegebene TAN in das dafür vorgesehene Feld ein

final_login = driver.find_element(By.XPATH, value = "//input[@id='derXpathdavon']")

actions = ActionChains(driver)
actions.click(final_login)
actions.perform()

download_csv = driver.find_element(By.XPATH, value = "//button[@id='downloadCSV']")#creates an object to download the transactions csv
actions = ActionChains(driver)
actions.click(download_csv)
actions.perform()


In [ ]:
#@title Filter downloaded CSV
#Code to actually filter csv by relevant transactions
import pandas as pd
import numpy as np
import re
import csv
import os











nameIn =input('Which .csv file would you like to input? Paste the entire name, with the .csv ending pls\n')

read_file = pd.read_csv (nameIn)
read_file.to_excel ('zwischendoc.xlsx', index = None, header=True) ##Wandelt .csv in .xlsx Dokument um


excel = pd.read_excel("zwischendoc.xlsx") #least erstellte .xlsx Datei ein als "excel"

df = pd.DataFrame(excel) #Wandelt "excel" ind dataframe "df" um
df['Booking text'] = df['Booking text'].replace(to_replace = "0", value = 'O', regex = True)#Ändert alle 0PG zu OPG, da diese trotz dem or Teil nicht erkannt werden
df['Booking text'] = df['Booking text'].astype(str) #Wandelt Text nach dem gefiltert werden soll in String um
df = df.loc[df['Booking text'].str.contains('OPG' or '0PG')] #Filtert anhand von String, je nachdem wo der Text OPG vorkommt##vielleicht noch auch nach 0PG filtern

df = df.loc[:,["Transaction date", "Booking text", "Amount"]] #gibt nur die drei Reihen aus

nameOut = input('What should the name of the output file be? REMEMBER, add .xlsx to the end\n') #User gibt Name von Outputdatei ein
os.remove("zwischendoc.xlsx")   ##soll Zwischendokument entfernen
df.to_excel(nameOut)#Speichert Dokument mit dem vom User gegebenen Namen ab